In [1]:
# python
# python
import sys
import importlib
# columnar analysis
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import processor
# local
sidm_path = str(sys.path[0]).split("/sidm")[0]
if sidm_path not in sys.path: sys.path.insert(1, sidm_path)
from sidm.tools import llpnanoaodschema, utilities
# always reload local modules to pick up changes during development
importlib.reload(llpnanoaodschema)
importlib.reload(utilities)

<module 'sidm.tools.utilities' from '/home/cms-jovyan/SIDM/sidm/tools/utilities.py'>

In [2]:
samples = [
    '2Mu2E_500GeV_5p0GeV_8p0mm',
]
fileset = utilities.make_fileset(samples, "llpNanoAOD_v2", max_files=1, location_cfg="signal_2mu2e_v10.yaml")
# create events collection from single file
fname = fileset[samples[0]]["files"][0]

In [3]:
factory_llp = NanoEventsFactory.from_root(
    fname,
    schemaclass=llpnanoaodschema.LLPNanoAODSchema,
)
events_llp = factory_llp.events()

/usr/local/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:205: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/usr/local/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:205: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(


In [4]:
pf = events_llp.Muon
dsa = events_llp.DSAMuon
print(dsa)
dsa.fields

[[DSAMuon, DSAMuon], [DSAMuon, DSAMuon, ... DSAMuon, DSAMuon], [DSAMuon, DSAMuon]]


['idx',
 'pt',
 'ptErr',
 'eta',
 'etaErr',
 'phi',
 'phiErr',
 'charge',
 'dxy',
 'dz',
 'vx',
 'vy',
 'vz',
 'chi2',
 'ndof',
 'trkNumPlanes',
 'trkNumHits',
 'trkNumDTHits',
 'trkNumCSCHits',
 'normChi2',
 'outerEta',
 'outerPhi',
 'dzPV',
 'dzPVErr',
 'dxyPVTraj',
 'dxyPVTrajErr',
 'dxyPVSigned',
 'dxyPVSignedErr',
 'ip3DPVSigned',
 'ip3DPVSignedErr',
 'dxyBS',
 'dxyBSErr',
 'dzBS',
 'dzBSErr',
 'dxyBSTraj',
 'dxyBSTrajErr',
 'dxyBSSigned',
 'dxyBSSignedErr',
 'ip3DBSSigned',
 'ip3DBSSignedErr',
 'displacedID',
 'muonMatch1',
 'muonMatch1idx',
 'muonMatch2',
 'muonMatch2idx',
 'muonMatch3',
 'muonMatch3idx',
 'muonMatch4',
 'muonMatch4idx',
 'muonMatch5',
 'muonMatch5idx',
 'muonDTMatch1',
 'muonDTMatch1idx',
 'muonDTMatch2',
 'muonDTMatch2idx',
 'muonDTMatch3',
 'muonDTMatch3idx',
 'muonCSCMatch1',
 'muonCSCMatch1idx',
 'muonCSCMatch2',
 'muonCSCMatch2idx',
 'muonCSCMatch3',
 'muonCSCMatch3idx',
 'nSegments',
 'nDTSegments',
 'nCSCSegments',
 'muonMatch1idxG',
 'muonMatch2idxG',
 

The following command will only work if you comment out [these lines](https://github.com/scikit-hep/coffea/blob/539c4b58961b7bae86038caf67f1c7db83dded3a/coffea/nanoevents/transforms.py#L129-L130) in your local coffea install. It would be better to change the type of the `muonMatchNidx` branches to `int32` (it's currently a float for some reason), but I haven't figured out how.

In [8]:
dsa.muonMatch1idxG

<Array [[4607182418800017408, ... ] type='4364 * var * int64[parameters={"__doc_...'>

In [21]:
print(dsa[1].pt)
print(pf[1].pt)
print(dsa[1].muonIdxG)
print(dsa[1].muonIdxG)
print(dsa[1].matched_muons.pt)

[5.92, 322, 1.62]
[212, 5.35]
[[3, 2, -1, -1, -1], [2, 3, -1, -1, -1], [3, 2, -1, -1, -1]]
[[3, 2, -1, -1, -1], [2, 3, -1, -1, -1], [3, 2, -1, -1, -1]]
[[197, 212, None, None, None], ... None, None, None], [197, 212, None, None, None]]


Great, it appears matched_muons allows me to access the muons that correspond to `muonMatchNidx` branches. Next I'll check what happens after applying a selection to the muons

In [22]:
events_llp.Muon = pf[pf.pt > 10]
print(events_llp.Muon[1].pt)
print(dsa[1].matched_muons.pt)

[212]
[[197, 212, None, None, None], ... None, None, None], [197, 212, None, None, None]]


In [23]:
print(pf.pt)

[[717, 135], [212, 5.35], [197, 21.2], ... [285, 3.7], [87.3, 76.7], [142, 118]]


Oh no, the global index is picking out muons in the next event when muons from the current event have been removed. Why doesn't this happen for other idx branches?
